In [14]:
from modelscope.msdatasets import MsDataset

# testing sample num: 311
ds = MsDataset.load(
    "ccmusic-database/Guzheng_Tech99",
    subset_name="eval",
    split="test",
    cache_dir="./__pycache__",
)

# mel = []
# for item in ds:
#     mel.append(mel)

2024-12-18 13:45:01,543 - modelscope - INFO - dataset_type: 4
d:\Anaconda3\lib\site-packages\modelscope\msdatasets\utils\hf_datasets_util.py:755: FutureWarning: The repository for ccmusic-database/Guzheng_Tech99 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccmusic-database/Guzheng_Tech99
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
d:\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [43]:
print(ds['train'])
# a = next
temp = ds['train']
a = next(iter(temp))
print(len(a['label']))
# m = a['label']
# print(a)
# m = a
# c = len(next(iter(ds))['label'])
# print(c)



Dataset({
    features: ['mel', 'cqt', 'chroma', 'label'],
    num_rows: 2486
})
7


In [17]:
import numpy as np 
temp = next(iter(ds))
# len(temp[0][0])
print(temp['mel'])
a = temp['chroma']
t = np.array(a)
print(t.shape)
l = temp['label']
len(l[0])
# ll = np.array(l)

# print(ll.shape)
# import numpy as np
# lab = a['mel']
# t = np.array(lab)
# print(lab)
# print(len(lab[6]))
# print(t.shape)

[[[-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [17.82135772705078], [40.481910705566406], [41.375526428222656], [33.78916931152344], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], [-0.03249843418598175], 

258

In [16]:
import torch
import torch.nn.functional as F
a = torch.randn(4, 3, 1, 200)
b = F.interpolate(a, size=(1,258), mode='bilinear', align_corners=False)

print(b.shape)

torch.Size([4, 3, 1, 258])


In [10]:
from torchvision.transforms import Compose, Resize, RandomAffine, ToTensor, Normalize
import torch
import numpy as np
img_size = 230
# t = torch.randn(3,210, 210)
t = np.random.randn(128, 210, 3)
print(t.shape)
t = t.transpose(2, 0, 1)
# compose = Compose(
#         [
#             Resize([img_size, img_size]),
#             RandomAffine(5),
#             ToTensor(),
#             Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#         ]
#     )
# a = compose(t)
# t = torch.randn(210, 210, 3).transpose(1, 2, 0)
t.shape

(128, 210, 3)


(3, 128, 210)

In [22]:
import torch.nn as nn
import torch
import torch.nn.functional as F
class Interpolate(nn.Module):
    def __init__(self, size=None, scale_factor=None, mode='bilinear', align_corners=False):
        super(Interpolate, self).__init__()
        self.size = size
        self.scale_factor = scale_factor
        self.mode = mode
        self.align_corners = align_corners

    def forward(self, x):
        return F.interpolate(x, size=self.size, scale_factor=self.scale_factor,
                             mode=self.mode, align_corners=self.align_corners)

upsample_module = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, None)), # F -> 1
            
            nn.ConvTranspose2d(512, 256, kernel_size=(1,4), stride=(1,2), padding=(0,1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),

            nn.ConvTranspose2d(256, 128, kernel_size=(1,4), stride=(1,2), padding=(0,1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),

            nn.ConvTranspose2d(128, 64, kernel_size=(1,4), stride=(1,2), padding=(0,1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),

            nn.ConvTranspose2d(64, 32, kernel_size=(1,4), stride=(1,2), padding=(0,1)),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),

            # input for Interp: [bsz, C, 1, T]
            Interpolate(size=(1, 258), mode='bilinear', align_corners=False),
            # classifier
            nn.Conv2d(32, 32, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 7, kernel_size=1) 
        )

ten = torch.randn(4, 512, 13, 13)
out = upsample_module(ten)
print(out.shape)
print(out.dim())

torch.Size([4, 7, 1, 258])
4


In [32]:
# metrics
preds = torch.randn(4, 7, 258)
labels = torch.randn(4, 7, 258)
threshold = 0
preds = (preds >= threshold).int()
labels = (labels >= threshold).int()
TP_per_frame = (preds & labels).sum(dim=1)  # Sum over classes, shape: [bsz, T]
FP_per_frame = (preds & ~labels).sum(dim=1)  # False positives, shape: [bsz, T]
FN_per_frame = (~preds & labels).sum(dim=1)  

epsilon = 1e-7  # To avoid division by zero
precision_per_frame = TP_per_frame / (TP_per_frame + FP_per_frame + epsilon)  # Shape: [bsz, T]
recall_per_frame = TP_per_frame / (TP_per_frame + FN_per_frame + epsilon)  # Shape: [bsz, T]
f1_per_frame = 2 * precision_per_frame * recall_per_frame / (precision_per_frame + recall_per_frame + epsilon)  # Shape: [bsz, T]

print(f1_per_frame.shape)

frame_f1 = f1_per_frame.mean()
print(frame_f1)


torch.Size([4, 258])
tensor(0.4504)


In [36]:
print(f'{frame_f1:.2f}')

0.45


In [21]:
from torchvision.models import squeezenet1_1
import torch.nn as nn

model = squeezenet1_1(pretrained=True)
print(model.forward.__code__)

Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to C:\Users\oasis/.cache\torch\hub\checkpoints\squeezenet1_1-b8a52dc0.pth


  0%|          | 0.00/4.73M [00:00<?, ?B/s]

<code object forward at 0x000002BD17E44710, file "d:\Anaconda3\lib\site-packages\torchvision\models\squeezenet.py", line 109>


# li dichucheng CODE
IPT_label shape: [IPT (7), cqt_time_steps]
s = 3*44100 / 512 = 258

其中
# length是0 padding后的长度，确保是s的整数倍
length = int(np.ceil((int(len(xdata) / s) + 1) * s))
若xdata shape: [258*2, F], 则length=774 = 258*2 + 258

# app用来padding 0 的
app : [258, F] 
# xdata是padding 0后的数据，time长度为s的整数倍，s是3秒片段。相当于不足3s的尾部被padding到3s
xdata : [774, F]

length / s 是

In [26]:
xdata = np.random.rand(258*2 + 16, 88)
s = 258
length = int(np.ceil((int(len(xdata) / s) + 1) * s))
app = np.zeros((length - xdata.shape[0], xdata.shape[1]))
xdata = np.concatenate((xdata, app), 0)
length / s

3.0

2.0